In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Transformations").getOrCreate()

data = [
    ("1234567890", "ACC123456", 150.75, "Purchase", "ABC Store", "New York, NY", "2024-08-25 14:32"),
    ("2345678901", "ACC234567", 500.00, "Withdrawal", "ATM", "Los Angeles, CA", "2024-08-25 09:15"),
    ("3456789012", "ACC345678", 2000.00, "Deposit", "Employer", "Chicago, IL", "2024-08-24 16:45"),
    ("4567890123", "ACC456789", 250.00, "Trade", "Stock Exchange", "Online", "2024-08-24 10:00"),
    ("5678901234", "ACC567890", 75.50, "Purchase", "XYZ Electronics", "Miami, FL", "2024-08-23 18:22"),
    ("5678901235", "ACC567890", 500, "Purchase", "Amazon", "Miami, FL", "2024-08-23 19:22"),
]

rdd = spark.sparkContext.parallelize(data)

In [3]:
def print_rdd(rdd):
  for row in rdd.collect():
    print(row)

In [ ]:
def categorize_amount(transaction):
    transaction_id, _, amount, *_ = transaction
    if amount < 100:
        category = "Low"
    elif amount < 1000:
        category = "Medium"
    else:
        category = "High"
    return (transaction_id, amount, category)

mapped_rdd = rdd.map(categorize_amount)

print_rdd(mapped_rdd)

('1234567890', 150.75, 'Medium')
('2345678901', 500.0, 'Medium')
('3456789012', 2000.0, 'High')
('4567890123', 250.0, 'Medium')
('5678901234', 75.5, 'Low')
('5678901235', 500, 'Medium')


In [ ]:
flat_mapped_rdd = rdd.flatMap(lambda x: x[4].split(" "))

print_rdd(flat_mapped_rdd)

ABC
Store
ATM
Employer
Stock
Exchange
XYZ
Electronics
Amazon


In [ ]:
filtered_rdd = rdd.filter(lambda x: x[2] > 100)

print_rdd(filtered_rdd)

('1234567890', 'ACC123456', 150.75, 'Purchase', 'ABC Store', 'New York, NY', '2024-08-25 14:32')
('2345678901', 'ACC234567', 500.0, 'Withdrawal', 'ATM', 'Los Angeles, CA', '2024-08-25 09:15')
('3456789012', 'ACC345678', 2000.0, 'Deposit', 'Employer', 'Chicago, IL', '2024-08-24 16:45')
('4567890123', 'ACC456789', 250.0, 'Trade', 'Stock Exchange', 'Online', '2024-08-24 10:00')
('5678901235', 'ACC567890', 500, 'Purchase', 'Amazon', 'Miami, FL', '2024-08-23 19:22')


In [ ]:
distinct_rdd = rdd.map(lambda x: x[1]).distinct()

print_rdd(distinct_rdd)

ACC234567
ACC345678
ACC456789
ACC567890
ACC123456


In [ ]:
sampled_rdd = rdd.sample(withReplacement=False, fraction=0.4)

print_rdd(sampled_rdd)

('2345678901', 'ACC234567', 500.0, 'Withdrawal', 'ATM', 'Los Angeles, CA', '2024-08-25 09:15')
('4567890123', 'ACC456789', 250.0, 'Trade', 'Stock Exchange', 'Online', '2024-08-24 10:00')
('5678901234', 'ACC567890', 75.5, 'Purchase', 'XYZ Electronics', 'Miami, FL', '2024-08-23 18:22')
('5678901235', 'ACC567890', 500, 'Purchase', 'Amazon', 'Miami, FL', '2024-08-23 19:22')


In [ ]:
coalesced_rdd = rdd.coalesce(2)

print_rdd(coalesced_rdd)

('1234567890', 'ACC123456', 150.75, 'Purchase', 'ABC Store', 'New York, NY', '2024-08-25 14:32')
('2345678901', 'ACC234567', 500.0, 'Withdrawal', 'ATM', 'Los Angeles, CA', '2024-08-25 09:15')
('3456789012', 'ACC345678', 2000.0, 'Deposit', 'Employer', 'Chicago, IL', '2024-08-24 16:45')
('4567890123', 'ACC456789', 250.0, 'Trade', 'Stock Exchange', 'Online', '2024-08-24 10:00')
('5678901234', 'ACC567890', 75.5, 'Purchase', 'XYZ Electronics', 'Miami, FL', '2024-08-23 18:22')
('5678901235', 'ACC567890', 500, 'Purchase', 'Amazon', 'Miami, FL', '2024-08-23 19:22')


In [ ]:
partitioned_rdd = rdd.repartition(4)

print_rdd(coalesced_rdd)

('1234567890', 'ACC123456', 150.75, 'Purchase', 'ABC Store', 'New York, NY', '2024-08-25 14:32')
('2345678901', 'ACC234567', 500.0, 'Withdrawal', 'ATM', 'Los Angeles, CA', '2024-08-25 09:15')
('3456789012', 'ACC345678', 2000.0, 'Deposit', 'Employer', 'Chicago, IL', '2024-08-24 16:45')
('4567890123', 'ACC456789', 250.0, 'Trade', 'Stock Exchange', 'Online', '2024-08-24 10:00')
('5678901234', 'ACC567890', 75.5, 'Purchase', 'XYZ Electronics', 'Miami, FL', '2024-08-23 18:22')
('5678901235', 'ACC567890', 500, 'Purchase', 'Amazon', 'Miami, FL', '2024-08-23 19:22')


In [ ]:
data_august = [
    ("1234567890", "ACC123456", 150.75, "Purchase", "ABC Store", "New York, NY", "2024-08-25 14:32"),
    ("2345678901", "ACC234567", 500.00, "Withdrawal", "ATM", "Los Angeles, CA", "2024-08-25 09:15")
]

data_september = [
    ("3456789012", "ACC345678", 2000.00, "Deposit", "Employer", "Chicago, IL", "2024-09-01 16:45"),
    ("4567890123", "ACC456789", 250.00, "Trade", "Stock Exchange", "Online", "2024-09-02 10:00")
]

rdd1 = spark.sparkContext.parallelize(data_august)
rdd2 = spark.sparkContext.parallelize(data_september)

union_rdd = rdd1.union(rdd2)
print_rdd(union_rdd)

('1234567890', 'ACC123456', 150.75, 'Purchase', 'ABC Store', 'New York, NY', '2024-08-25 14:32')
('2345678901', 'ACC234567', 500.0, 'Withdrawal', 'ATM', 'Los Angeles, CA', '2024-08-25 09:15')
('3456789012', 'ACC345678', 2000.0, 'Deposit', 'Employer', 'Chicago, IL', '2024-09-01 16:45')
('4567890123', 'ACC456789', 250.0, 'Trade', 'Stock Exchange', 'Online', '2024-09-02 10:00')


In [ ]:
data_august = [
    ("1234567890", "ACC123456", 150.75, "Purchase", "ABC Store", "New York, NY", "2024-08-25 14:32"),
    ("2345678901", "ACC234567", 500.00, "Withdrawal", "ATM", "Los Angeles, CA", "2024-08-25 09:15"),
    ("3456789012", "ACC345678", 2000.00, "Deposit", "Employer", "Chicago, IL", "2024-08-24 16:45")
]

data_high_value = [
    ("2345678901", "ACC234567", 500.00, "Withdrawal", "ATM", "Los Angeles, CA", "2024-08-25 09:15"),
    ("3456789012", "ACC345678", 2000.00, "Deposit", "Employer", "Chicago, IL", "2024-08-24 16:45"),
    ("4567890123", "ACC456789", 250.00, "Trade", "Stock Exchange", "Online", "2024-09-02 10:00")
]

rdd1 = spark.sparkContext.parallelize(data_august)
rdd2 = spark.sparkContext.parallelize(data_high_value)

intersection_rdd = rdd1.intersection(rdd2)
print_rdd(intersection_rdd)

('3456789012', 'ACC345678', 2000.0, 'Deposit', 'Employer', 'Chicago, IL', '2024-08-24 16:45')
('2345678901', 'ACC234567', 500.0, 'Withdrawal', 'ATM', 'Los Angeles, CA', '2024-08-25 09:15')


In [ ]:
grouped_rdd = rdd.map(lambda x: (x[1], x)).groupByKey()

print_rdd(grouped_rdd)

('ACC234567', <pyspark.resultiterable.ResultIterable object at 0x7d67341fe980>)
('ACC345678', <pyspark.resultiterable.ResultIterable object at 0x7d67341fcaf0>)
('ACC456789', <pyspark.resultiterable.ResultIterable object at 0x7d67341fd270>)
('ACC567890', <pyspark.resultiterable.ResultIterable object at 0x7d67341fed10>)
('ACC123456', <pyspark.resultiterable.ResultIterable object at 0x7d67341fe3e0>)


In [ ]:
reduced_rdd = rdd.map(lambda x: (x[1], x[2])).reduceByKey(lambda a, b: a + b)

print_rdd(reduced_rdd)

('ACC234567', 500.0)
('ACC345678', 2000.0)
('ACC456789', 250.0)
('ACC567890', 575.5)
('ACC123456', 150.75)


In [ ]:
sorted_rdd = rdd.map(lambda x: (x[1], x)).sortByKey()

print_rdd(sorted_rdd)

('ACC123456', ('1234567890', 'ACC123456', 150.75, 'Purchase', 'ABC Store', 'New York, NY', '2024-08-25 14:32'))
('ACC234567', ('2345678901', 'ACC234567', 500.0, 'Withdrawal', 'ATM', 'Los Angeles, CA', '2024-08-25 09:15'))
('ACC345678', ('3456789012', 'ACC345678', 2000.0, 'Deposit', 'Employer', 'Chicago, IL', '2024-08-24 16:45'))
('ACC456789', ('4567890123', 'ACC456789', 250.0, 'Trade', 'Stock Exchange', 'Online', '2024-08-24 10:00'))
('ACC567890', ('5678901234', 'ACC567890', 75.5, 'Purchase', 'XYZ Electronics', 'Miami, FL', '2024-08-23 18:22'))
('ACC567890', ('5678901235', 'ACC567890', 500, 'Purchase', 'Amazon', 'Miami, FL', '2024-08-23 19:22'))


In [ ]:
account_details = [
    ("ACC123456", "John Doe"),
    ("ACC234567", "Jane Smith"),
    ("ACC345678", "Alice Johnson"),
    ("ACC456789", "Bob Brown"),
    ("ACC567890", "Charlie Green")
]

account_details_rdd = spark.sparkContext.parallelize(account_details)

transactions_rdd = rdd.map(lambda x: (x[1], x))

joined_rdd = transactions_rdd.join(account_details_rdd)

print_rdd(joined_rdd)

('ACC234567', (('2345678901', 'ACC234567', 500.0, 'Withdrawal', 'ATM', 'Los Angeles, CA', '2024-08-25 09:15'), 'Jane Smith'))
('ACC345678', (('3456789012', 'ACC345678', 2000.0, 'Deposit', 'Employer', 'Chicago, IL', '2024-08-24 16:45'), 'Alice Johnson'))
('ACC456789', (('4567890123', 'ACC456789', 250.0, 'Trade', 'Stock Exchange', 'Online', '2024-08-24 10:00'), 'Bob Brown'))
('ACC567890', (('5678901234', 'ACC567890', 75.5, 'Purchase', 'XYZ Electronics', 'Miami, FL', '2024-08-23 18:22'), 'Charlie Green'))
('ACC567890', (('5678901235', 'ACC567890', 500, 'Purchase', 'Amazon', 'Miami, FL', '2024-08-23 19:22'), 'Charlie Green'))
('ACC123456', (('1234567890', 'ACC123456', 150.75, 'Purchase', 'ABC Store', 'New York, NY', '2024-08-25 14:32'), 'John Doe'))


In [4]:
all_transactions = rdd.collect()
for transaction in all_transactions:
    print(transaction)

('1234567890', 'ACC123456', 150.75, 'Purchase', 'ABC Store', 'New York, NY', '2024-08-25 14:32')
('2345678901', 'ACC234567', 500.0, 'Withdrawal', 'ATM', 'Los Angeles, CA', '2024-08-25 09:15')
('3456789012', 'ACC345678', 2000.0, 'Deposit', 'Employer', 'Chicago, IL', '2024-08-24 16:45')
('4567890123', 'ACC456789', 250.0, 'Trade', 'Stock Exchange', 'Online', '2024-08-24 10:00')
('5678901234', 'ACC567890', 75.5, 'Purchase', 'XYZ Electronics', 'Miami, FL', '2024-08-23 18:22')
('5678901235', 'ACC567890', 500, 'Purchase', 'Amazon', 'Miami, FL', '2024-08-23 19:22')


In [5]:
transaction_count = rdd.count()
print(transaction_count)

6


In [8]:
three_transactions = rdd.take(3)

print(three_transactions)

[('1234567890', 'ACC123456', 150.75, 'Purchase', 'ABC Store', 'New York, NY', '2024-08-25 14:32'), ('2345678901', 'ACC234567', 500.0, 'Withdrawal', 'ATM', 'Los Angeles, CA', '2024-08-25 09:15'), ('3456789012', 'ACC345678', 2000.0, 'Deposit', 'Employer', 'Chicago, IL', '2024-08-24 16:45')]


In [9]:
top_three_transactions = rdd.top(3, key=lambda x: x[2])

print(top_three_transactions)

[('3456789012', 'ACC345678', 2000.0, 'Deposit', 'Employer', 'Chicago, IL', '2024-08-24 16:45'), ('2345678901', 'ACC234567', 500.0, 'Withdrawal', 'ATM', 'Los Angeles, CA', '2024-08-25 09:15'), ('5678901235', 'ACC567890', 500, 'Purchase', 'Amazon', 'Miami, FL', '2024-08-23 19:22')]


In [10]:
total_amount = rdd.map(lambda x: x[2]).reduce(lambda a, b: a + b)

print(total_amount)

3476.25


In [11]:
rdd.saveAsTextFile("output/transactions")